In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, ElasticNetCV, LassoCV, RidgeCV, Lasso, SGDRegressor
from data_cleaner import eliza_cleaning, eliza_fillna
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, Normalizer, StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import explained_variance_score as evs # evaluation metric
from sklearn.metrics import r2_score as r2 # evaluation metric
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error
from pickle import load

In [24]:
raw_datas = pd.read_csv('https://raw.githubusercontent.com/JulienAlardot/challenge-collecting-data/main/Data/database.csv')
datas = eliza_cleaning(raw_datas)
datas = eliza_fillna(datas)
median = pd.read_csv('median.csv')
post = pd.read_csv('post_codes.csv', sep=';')
median['Gemeente'] = median['Gemeente'].str.lower()
post['Commune Principale'] = post['Commune principale'].str.lower()
median_with_post = median.merge(post[['Code postal', 'Commune Principale']], how='left', left_on='Gemeente', right_on='Commune Principale')
median_with_post = median_with_post.groupby('Gemeente').median()
median_with_post['Mediaanprijs 2020'].fillna(median_with_post['Mediaanprijs 2019'], inplace=True)
median_with_post['Mediaanprijs 2020'].fillna(median_with_post['Mediaanprijs 2018'], inplace=True)
median_with_post.sort_values(by='Code postal', inplace=True)
median_with_post.fillna(method='bfill', inplace=True)
median_with_post.reset_index(inplace=True)
median = median.merge(median_with_post[['Gemeente', 'Mediaanprijs 2020']], on='Gemeente')
median_with_post = median.merge(post[['Code postal', 'Commune Principale']], how='left', left_on='Gemeente', right_on='Commune Principale')
median_prices = median_with_post[['Code postal', 'Mediaanprijs 2020_y']]
median_prices.columns = ['postal_code', 'median_price']
median_prices = median_prices.groupby('postal_code').mean()
median_prices.reset_index(inplace=True)
median_prices['postal_code'] = median_prices['postal_code'].astype('int64')
datas = datas.merge(median_prices, how='left', left_on='locality', right_on='postal_code')
datas.drop('postal_code', inplace=True, axis=1)
datas.sort_values(by='locality', ascending=False, inplace=True)
datas['median_price'].fillna(method='ffill', inplace=True)
datas.sort_index(inplace=True)
col = datas.columns
col = [col[0]]+[col[-1]]+list(col[1:-1])
datas = datas[col]

In [25]:
y = datas.pop('price')
X = datas

In [26]:
pipe = load(open('preprocessor.pkl', 'rb'))
model_xgb = xgb.XGBRegressor()
model_xgb.load_model("model.json")


In [28]:
X = pipe.transform(X)

preds = model_xgb.predict(X)
rmse = np.sqrt(mean_squared_error( y, np.exp(preds)))
print("RMSE: %f" % (rmse))
print("R2 score: ", model_xgb.score(X,  np.log(y)))

RMSE: 173483.202269
R2 score:  0.8005509549923464
